# TESS bandpass and cloudy and cloud-free model atmosphere spectra

In this notebook we compare cloudy and cloud-free models to the TESS bandpass to make a figure for the TESS Science Conference and the paper.

The next step after that will be to integrate the cloudy and clear models to produce synthetic photometry scaling relations that could hypothetically inform the filling factor of clouds.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import astropy.units as u
import numpy as np
import specutils
from specutils import Spectrum1D
%config InlineBackend.figure_format = 'retina'

In [ ]:
plt.rcParams['figure.facecolor'] = 'white'

### Read in the TESS bandpass

In [ ]:
#! wget https://heasarc.gsfc.nasa.gov/docs/tess/data/tess-response-function-v2.0.csv
#! mv tess-response-function-v2.0.csv ../../data/TESS/
#! head ../../data/TESS/tess-response-function-v2.0.csv

In [ ]:
df_TESS = pd.read_csv('../../data/TESS/tess-response-function-v2.0.csv', 
                      skiprows=7, names=['wl_nm', 'transmission'])
df_TESS.plot('wl_nm', 'transmission');

In [ ]:
tess_bandpass = Spectrum1D(spectral_axis=(df_TESS.wl_nm.values*u.nm).to(u.micron), 
                           flux=df_TESS.transmission.values*u.dimensionless_unscaled)

### Read in the models, convert to common units

The models use different units for flux.  We will convert them.

In [ ]:
fn ='/home/gully/libraries/raw/morley_clouds_20210322/ldwarf_specs/t1300g178f2_m0.0_co1.0.spec'
df_cloudy = pd.read_csv(fn, skiprows=[0,1], delim_whitespace=True, names=['wl_um','flux']
                      ).sort_values('wl_um').reset_index(drop=True)

In [ ]:
df_cloud_free = pd.read_csv('/home/gully/libraries/raw/marley/sp_t1300g178nc_m0.0', 
                    skiprows=[0,1], delim_whitespace=True, names=['wl_um','flux']
                   ).sort_values('wl_um').reset_index(drop=True)

In [ ]:
marley_unit = u.erg/u.cm**2/u.s/u.Hz
morley_unit = u.Watt/u.m**2/u.m

In [ ]:
morley_flux_w_units = (df_cloudy.flux.values*morley_unit)
morley_wavelength = df_cloudy.wl_um.values*u.micron

morley_cgs = morley_flux_w_units.to(marley_unit, equivalencies=u.spectral_density(morley_wavelength))

In [ ]:
cloudy_native = Spectrum1D(spectral_axis=df_cloudy.wl_um.values*u.micron, flux=morley_cgs)

In [ ]:
cloud_free_native = Spectrum1D(spectral_axis=df_cloud_free.wl_um.values*u.micron, 
                               flux=df_cloud_free.flux.values*marley_unit)

### Resample the spectra to the TESS bandpass

In [ ]:
from specutils.manipulation import FluxConservingResampler, LinearInterpolatedResampler

In [ ]:
resampler = FluxConservingResampler(extrapolation_treatment='nan_fill')

In [ ]:
%%capture
cloud_free = resampler(cloud_free_native, tess_bandpass.spectral_axis)
cloudy = resampler(cloudy_native, tess_bandpass.spectral_axis)

### Only keep the non-zero bandpass

In [ ]:
plt.plot(tess_bandpass.wavelength, tess_bandpass.flux)
plt.yscale('log')

In [ ]:
mask = tess_bandpass.flux.value > 0

In [ ]:
min_wl = tess_bandpass.wavelength[mask].value.min()*tess_bandpass.wavelength.unit
max_wl = tess_bandpass.wavelength[mask].value.max()*tess_bandpass.wavelength.unit

In [ ]:
tess_bandpass = tess_bandpass[min_wl:max_wl]
cloudy = cloudy[min_wl:max_wl]
cloud_free = cloud_free[min_wl:max_wl]

Ok, our resampled spectrum should resemble the other.

In [ ]:
import seaborn as sns

In [ ]:
sns.set_context('talk')

In [ ]:
marley_unit.to_string('latex')

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(cloudy.wavelength, cloudy.flux, label='Cloudy');
plt.plot(cloud_free.wavelength, cloud_free.flux, label='Cloud-free');
plt.fill_between(tess_bandpass.wavelength, 
                 tess_bandpass.flux*np.median(cloudy.flux.value)*50, 
                 label='TESS Bandpass', color='#bdc3c7', alpha=0.5)
#plt.axvline(8340, linestyle='dashed', color='k', alpha=0.3)
#plt.axvline(10100, linestyle='dashed', color='k', alpha=0.3)
plt.legend(loc='best');
plt.xlabel('$\lambda (\AA)$')
plt.ylabel(r'$f_\nu$ ('+marley_unit.to_string('latex')+')');
plt.savefig('../../figures/TESS_bandpass_sonora_cloudy.png', dpi=300, bbox_inches='tight')

### Weight by the TESS bandpass

In [ ]:
scalar = cloudy.flux.mean()

In [ ]:
cloudy_weighted = cloudy * tess_bandpass / scalar
cloud_free_weighted = cloud_free * tess_bandpass / scalar

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(cloudy_weighted.wavelength, cloudy_weighted.flux, label='Cloudy');
plt.plot(cloud_free_weighted.wavelength, cloud_free_weighted.flux, label='Cloud-free');
plt.fill_between(tess_bandpass.wavelength, 
                 tess_bandpass.flux*2, 
                 label='TESS Bandpass', color='#bdc3c7', alpha=0.3)
plt.legend(loc='best');
plt.xlabel('$\lambda (\AA)$')
plt.ylabel('Bandpass weighted flux');

In [ ]:
plt.step(cloudy_weighted.wavelength, 
         cloudy_weighted.flux.value.cumsum()/cloudy_weighted.flux.value.sum(), 
         label='Cloudy');
plt.axhline(0.10, color='r')
plt.axhline(0.90, color='r')

In [ ]:
vec = cloudy_weighted.flux.value.cumsum()/cloudy_weighted.flux.value.sum()

In [ ]:
cloudy_weighted.wavelength[vec > 0.1]

In [ ]:
cloudy_weighted.wavelength[vec > 0.9]

In [ ]:
df_nc.wl_um.diff().median()*10_000, df_clouds.wl_um.diff().median()*10_000

In [ ]:
plt.step(df_clouds.wl_um, df_clouds.flux_cgs, label='Cloudy')
plt.step(df_nc.wl_um, df_nc.flux_cgs, label='Cloud-free')
plt.title(r'$T_{\mathrm{eff}}$ = 1000 K, $g=178$ m/s$^2$')
plt.yscale('log')
plt.xscale('linear')
plt.legend(loc='best');

Awesome!  That looks right!  On average, the clouds are brighter than the cloud-free atmosphere, for the same sized patch.

Let's overplot for just the range of the above IGRINS order.

In [ ]:
plt.step(df.wl, df.data, label='IGRINS $m=110$')
plt.step(df.wl, df.model_composite, label='Sonora cloud-free \n$T=1350$ K, $\log{g}=4.5$, $v\sin{i}=\;30\;$km/s')
plt.legend();
plt.xlabel('$\lambda \; (\AA)$')
plt.ylabel('$\propto$ Flux')
plt.ylim(0)
plt.title('Luhman 16 quicklook')
plt.savefig('../../figures/Luhman16_quicklook.png', dpi=300, bbox_inches='tight')

In [ ]:
from scipy.ndimage import gaussian_filter1d

In [ ]:
gaussian_filter1d(df_nc.flux_cgs, sigma=25)

In [ ]:
plt.figure(figsize=(15, 6))
plt.step(df.wl/10000, df.data*1e-6, label='IGRINS $m=110$', color='k')

smoothed_cloudy = gaussian_filter1d(df_clouds.flux_cgs, sigma=2)
smoothed_nocloud = gaussian_filter1d(df_nc.flux_cgs, sigma=11)
ff = 0.4
#net = ff*smoothed_cloudy + (1-ff)*smoothed_nocloud

#plt.step(df_nc.wl_um, net, label='Composite ($f=40$)')

plt.step(df_nc.wl_um, (1-ff)*smoothed_nocloud, label='Cloud-free')
plt.step(df_clouds.wl_um, ff*smoothed_cloudy, label='Cloudy')
plt.title(r'$T_{\mathrm{eff}}$ = 1000 K, $g=178$ m/s$^2$')
plt.legend(loc='best');
plt.xlim(1.6220, 1.64000)

In [ ]:
from muler.igrins import IGRINSSpectrum

In [ ]:
reduced_fns = glob.glob('../../data/IGRINS/originals/GS-2021A-DD-104/*/reduced/SDCK*.spec_a0v.fits')

In [ ]:
spec1 = IGRINSSpectrum(file=reduced_fns[1], order=10).normalize()

In [ ]:
spec1

In [ ]:
ax = spec1.plot()
plt.figure(figsize=(15, 6))
#plt.step(df.wl/10000, df.data*1e-6, label='IGRINS $m=110$', color='k')

smoothed_cloudy = gaussian_filter1d(df_clouds.flux_cgs, sigma=2)
smoothed_nocloud = gaussian_filter1d(df_nc.flux_cgs, sigma=12)
ff = 0.4
#net = ff*smoothed_cloudy + (1-ff)*smoothed_nocloud

#plt.step(df_nc.wl_um, net, label='Composite ($f=40$)')

plt.step(df_nc.wl_um, (1-ff)*smoothed_nocloud, label='Cloud-free')
plt.step(df_clouds.wl_um, ff*smoothed_cloudy, label='Cloudy')
plt.title(r'$T_{\mathrm{eff}}$ = 1000 K, $g=178$ m/s$^2$')
plt.legend(loc='best');
plt.xlim(2.240, 2.2600)